# Inports

Usual imports.


In [1]:
import numpy as np
import math
import pygame

# Constants

Important constants for the game GO to improve code readability.

In [11]:
black = -1
white = 1
empty = 0

# Classes

Definitions of useful classes

In [9]:
class Board:
    game_board = None
    turn = 0
    size = None
    
    def __init__(self, size):
        self.game_board = np.zeros((size,size), dtype=np.int8)
        self.size = size
        
    def get_situation(self):
        return self.game_board
    
    def get_board_size(self):
        return self.size
    
    def get_turn(self):
        return self.turn

# Game Functions

#  Main

In [10]:
def main():
    board = Board(13)
    print(board.get_situation().shape)
    
    

if __name__ == "__main__":
    main()

(13, 13)
